# Autoloading

Loading graph visualisation settings.

In [ ]:
%%capture "Remove this line to see debug information"
%%graph_notebook_vis_options
{
  "edges": {
    "smooth": {
      "enabled": true,
      "type": "dynamic"
    },
    "arrows": {
      "to": {
        "enabled": true,
        "type": "arrow"
      }
    }
  }
}

# Initial Setup

## Get a view of all Ingested Cluster

Retrieve all the current cluster ingested in KubeHound with the associated runID with the number of nodes. This numbers can be used to get a clue of the size of the cluster and also identify if an ingestion did not complete.

In [ ]:
%%gremlin -d class -g critical -le 50 -p inv,oute

kh.nodes()
    .groupCount()
    .by(project('cluster','runID')
         .by('cluster').by('runID'))
    .unfold()
    .limit(1000)

## Setting your run_id/cluster

Set which runID you want to use. The variable are being shared with all users of the instance, so we advise to make a uniq string for your user `runID_yourid` to avoid any conflict.

In [ ]:
%%gremlin -d class -g critical -le 50 -p inv,oute

graph.variables()
    .set('runID_yourid','01htdgjj34mcmrrksw4bjy2e94')

# Container escapes

List all containers which are vulnerable to container escape to the node. 

## Identify the vulnerable containers

The goal of this list is to identify images vulnerable to container escape. It will list all the containers and remove duplicate entry that share the same `namespace`, `app`, `team` and `image` labels.

In [ ]:
%%gremlin -d class -g critical -le 50 -p inv,oute

kh.containers()
    .has("runID", graph.variables().get('runID_yourid').get().trim())
    .where(
        repeat(
          outE().inV().simplePath()  // Building the path from one vertex to another
        ).until(
            has(label, "Node")       // Stop when meeting a critical asset
            .or().loops().is(10)     // Stop after X iteration
        ).has(label, "Node")         // Keep only path ending with a critical asset
        .limit(1)
    )
    .dedup().by("image")
    .valueMap("namespace","app","team","image")
    .limit(1000)

If the list above is still too big to handle you can start with a more narrow view. The following list give a more abstract view to get deduplicated list of vulnerable `app`/`namespace`.

If the k8s label `app` is not set properly, you scope it by `namespace`.

In [ ]:
%%gremlin -d class -g critical -le 50 -p inv,oute

kh.containers()
    .has("runID", graph.variables().get('runID_yourid').get().trim())
    .where(
        repeat(
          outE().inV().simplePath()  // Building the path from one vertex to another
        ).until(
            has(label, "Node")       // Stop when meeting a critical asset
            .or().loops().is(10)     // Stop after X iteration
        ).has(label, "Node")         // Keep only path ending with a critical asset
        .limit(1)
    )
    .dedup()
        .by("namespace")
        .by("app")
    .valueMap("namespace","app")
    .limit(1000)

The goal here is to extract a list of apps for which you accept the risk for XYZ reason, to ignore them in queries. You can set this exclude list of `app` or `namespace` using gremlin variables in the following cell:

In [ ]:
%%gremlin -d class -g critical -le 50 -p inv,oute

graph.variables()
    .set('containerEscape_whiteListedApp_yourid',['WHITELISTED_APP1', "WHITELISTED_APP2"])

graph.variables()
    .set('containerEscape_whiteListedNamespace_yourid',['NAMESPACE1', "NAMESPACE2"])

To filter them out, add the following `.not(has(...whiteListedApp...).or(...whiteListedNamespace...)` block at the start of the Gremlin queries

In [ ]:
%%gremlin -d class -g critical -le 50 -p inv,oute

kh.containers()
    .has("runID", graph.variables().get('runID_yourid').get().trim())
    .not(
        has("app", within(graph.variables().get('containerEscape_whiteListedApp_yourid').get()))
        .or().has("namespace", within(graph.variables().get('containerEscape_whiteListedNamespace_yourid').get()))
    )

## Manual investigation for each app/namespace

From the above list, you can manually investigate each vulnerable `app`/`namespace`. To proceed with the investigation, just copy/paste the name of the vulnerable app (replace `VULNERABLE_APP` by the targetted app).

In [ ]:
%%gremlin -d class -g critical -le 50 -p inv,oute

graph.variables()
    .set('containerEscape_vulnApp_yourid','VULNERABLE_APP')

### Listing all attack paths from a particular app

The following gremlin request will **list all container escapes for the selected app**. We add a limit(1000) to avoid having huge graph.

In [ ]:
%%gremlin -d class -g critical -le 50 -p inv,oute

kh.containers()
    .has("runID", graph.variables().get('runID_yourid').get().trim())
    .has("app",graph.variables().get('containerEscape_vulnApp_yourid').get().trim())
    .repeat(
      outE().inV().simplePath()   // Building the path from one vertex to another
    ).until(
        has(label, "Node")        // Stop when meeting a critical asset
        .or().loops().is(10)      // Stop after X iteration
    ).has(label, "Node")          // Keep only path ending with a critical asset
    .path().by(elementMap())
    .limit(1000)

The last view can already be quite overwhelming, even if it might not be an exhaustive view (as we capped the result with `limit(1000)`). Increasing the limit will not solve the issue as it will become humanly unreadable. 

### Listing all attack path deduplicated by app from a particular app 

One way to solve it is to generate an **overall view to understand the attack path**. This view will strip any specific information (image, ids, ...) and keep only 3 labels:
* the `app` label which specify what is associated application
* the `class` of the object (node, pod, role, ...) 
* if the resource is `critical`. 

For instance, this will remove any replicatset duplication.

In [ ]:
%%gremlin -d class -g critical -le 50 -p inv,oute

kh.containers()
    .has("runID", graph.variables().get('runID_yourid').get().trim())
    .has("app",graph.variables().get('containerEscape_vulnApp_yourid').get().trim())
    .repeat(
      outE().inV().simplePath()   // Building the path from one vertex to another
    ).until(
        has(label, "Node")        // Stop when meeting a critical asset
        .or().loops().is(10)      // Stop after X iteration
    ).has(label, "Node")          // Keep only path ending with a critical asset
    .path()
    .by(valueMap("app", "class","critical").with(WithOptions.tokens,WithOptions.labels))
    .dedup()
    .limit(1000)

### Listing all attack path deduplicated by label/type from a particular app 

Sometimes, the previous view is still too big and return too many elements to be easily processable. So, to get an even widder picture, we can deduplicate the attack paths by k8s resource type only. This show the interaction from one type (endpoints/containers/nodes/...) to try to understand the bigger picture.

In [ ]:
%%gremlin -d class -g critical -le 50 -p inv,oute

kh.containers()
    .has("runID", graph.variables().get('runID_yourid').get().trim())
    .has("app",graph.variables().get('containerEscape_vulnApp_yourid').get().trim())
    .repeat(
      outE().inV().simplePath()   // Building the path from one vertex to another
    ).until(
        has(label, "Node")        // Stop when meeting a critical asset
        .or().loops().is(10)      // Stop after X iteration
    ).has(label, "Node")          // Keep only path ending with a critical asset
    .path().by(label())
    .dedup()
    .limit(1000)

## Global view using the whitelisted approach

We are reusing the same queries as previously but instead of iterating over each app, we take the problem more globaly. This approach can be quicker but needs to have a smaller or secure cluster.

### Listing all attack paths (except the whitelisted one)

In [ ]:
%%gremlin -d class -g critical -le 50 -p inv,oute

kh.containers()
    .has("runID", graph.variables().get('runID_yourid').get().trim())
    .not(
        has("app", within(graph.variables().get('containerEscape_whiteListedApp_yourid').get()))
        .or().has("namespace", within(graph.variables().get('containerEscape_whiteListedNamespace_yourid').get()))
    )
    .repeat(
      outE().inV().simplePath()   // Building the path from one vertex to another
    ).until(
        has(label, "Node")        // Stop when meeting a critical asset
        .or().loops().is(10)      // Stop after X iteration
    ).has(label, "Node")          // Keep only path ending with a critical asset
    .path().by(elementMap())
    .limit(1000)

### Listing all attack path deduplicated by app (except the whitelisted one)

In [ ]:
%%gremlin -d class -g critical -le 50 -p inv,oute

kh.containers()
    .has("runID", graph.variables().get('runID_yourid').get().trim())
    .not(
        has("app", within(graph.variables().get('containerEscape_whiteListedApp_yourid').get()))
        .or().has("namespace", within(graph.variables().get('containerEscape_whiteListedNamespace_yourid').get()))
    )
    .repeat(
      outE().inV().simplePath()   // Building the path from one vertex to another
    ).until(
        has(label, "Node")        // Stop when meeting a critical asset
        .or().loops().is(10)      // Stop after X iteration
    ).has(label, "Node")          // Keep only path ending with a critical asset
    .path()
    .by(valueMap("app", "class","critical").with(WithOptions.tokens,WithOptions.labels))
    .dedup()
    .limit(1000)

### Listing all attack path deduplicated by label/type (except the whitelisted one)

In [ ]:
%%gremlin -d class -g critical -le 50 -p inv,oute

kh.containers()
    .has("runID", graph.variables().get('runID_yourid').get().trim())
    .not(
        has("app", within(graph.variables().get('containerEscape_whiteListedApp_yourid').get()))
        .or().has("namespace", within(graph.variables().get('containerEscape_whiteListedNamespace_yourid').get()))
    )
    .repeat(
      outE().inV().simplePath()   // Building the path from one vertex to another
    ).until(
        has(label, "Node")        // Stop when meeting a critical asset
        .or().loops().is(10)      // Stop after X iteration
    ).has(label, "Node")          // Keep only path ending with a critical asset
    .path().by(label())
    .dedup()
    .limit(1000)